# Neutral Review Classification

## Overview

Consumer reviews has historically been one of the best ways for business owners to understand more fully the needs and wants of their consumers, thus being an important tool for businesses to leverage in order to create a complete and enjoyable experience that encompasses their consuemr base. One important tool for this is business intelligence. However, business intelligence suites are generally expensive and inflexible, making it difficult for smaller businesses that are not largely scaled to gain a sufficient amount of value from these business suites to justify spending the amount it costs to utilize them. Thus, I wanted to create a business intelligence tool that could add value to consumer reviews that may not seem informationally valuable at first glance.

#### The Neutral Feedback Problem: 
Generally speaking, 4 and 5 star reviews are positive consumer experiences, and the language in these reviews reflect that with highlights and recommendations to others. On the other end of the spectrum, 1 and 2 star reviews are generally negative consumer experiences, and the language in these reviews are facets of the experience that consumers think should be improved. This can more easily be thought of as: 4-5 star reviews - highlights, and 1-2 star reviews - improvements needed. However, in the middle ground, a place of difficult interpretability, are the neutral 3 star reviews. These reviews typically have a middle ground between "highlights" and "improvements needed", thus making it difficult to quickly glean any information, unless a human manually goes through and classifies the language in the context of the review.

## Business Case

The goal of this project is to add informational value to the difficult-to-interpret reviews by classifying them as positive or negative, hopefully offering a quick way for small business owners to gain value from these neutral reviews.

## Start of Data Cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from collections import defaultdict
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, f1_score
import string
import pickle
import spacy
from nltk.util import ngrams
import nltk, re, string, collections

In [ ]:
df = pd.read_csv('data/smaller/yelp_review.csv')
df.head()

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df[df['stars'] == 3]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().value_counts()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.info()

# Data Understanding

Some EDA and visualizations here to give us a better idea of the data we're working with

In [ ]:
# Import list of stopwords from SpaCy
from spacy.lang.en.stop_words import STOP_WORDS
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
# Create a function to tokenize the text of the articles
punctuation = [*string.punctuation , *[str(x) for x in list(range(0,10))]]
nlp = spacy.load('en_core_web_sm')
def normalize(text):
    text = ''.join([x for x in text if x not in punctuation])
    toks = nlp(text)
    toks = [word.lemma_.lower().strip() for word in toks if word.pos_ != 'PRON']
    toks = [word for word in toks if word not in stop_words]
    return ' '.join(toks)

### Data Visualization

In [ ]:
eda = df.sample(500000, random_state = 42)

In [ ]:
eda.head()

#### Word Cloud

In [ ]:
# Apply tokenization function to the real articles. Create a new column for the processed articles
eda['processed_articles'] = eda['text'].map(lambda x: normalize(x))

In [ ]:
true_text = eda.processed_articles

In [ ]:
filename = 'pickle/true_text'
pickle.dump(true_text, open(filename, 'wb'))

In [ ]:
true_text = pickle.load(open('pickle/true_text', 'rb'))

In [ ]:
# Create a list of all of the tokenized words
true_list = []
for x in true_text:
    true_list.append(x)

In [ ]:
# Create and generate a word cloud image:
wordcloud = WordCloud().generate(str(true_list))

# Display the generated image:
plt.figure(figsize = (15, 15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('images/wordcloud.png')
plt.show()

#### Frequency Distribution

In [ ]:
# Create a new list of stopwords
stopwords = ['m', 've', 'nt', '``', 's', 'c', "'", ",", "t", "l", 'j', '...', ":", '0', '1', '2', 'couldn', 'wouldn', 'isn', 'aren', 'shouldn', 'don', 'doesn', 'didn']
# Create a new tokenized list to use in the frequency distribution tables
true_tokenized_word = nltk.word_tokenize(str(true_list))

In [ ]:
# Iterate through new tokenized list and remove additional stopwords.
true_tokenized_word = [word for word in true_tokenized_word if word not in stopwords]

In [ ]:
# Create frequency distribution for the words in the real articles
true_fdist=nltk.FreqDist(true_tokenized_word)

In [ ]:
print(true_fdist.most_common(20))

In [ ]:
# Plot frequency distribution for the 20 most common words
true_fdist.plot(20)
plt.savefig('images/freq_dist.png')
plt.show()

#### Bigrams

In [ ]:
# Get a list of all the bi-grams
true_bigrams = ngrams(true_tokenized_word, 2)

In [ ]:
# List 20 most common bigrams
true_bigrams_freq = collections.Counter(true_bigrams)
true_bigrams_freq.most_common(20)

In [ ]:
true_bigram_fdist=nltk.FreqDist(true_bigrams_freq)

In [ ]:
true_bigram_fdist.plot(20)
plt.savefig('images/bigram_freq.png')
plt.show()

#### Star Distribution

In [ ]:
list(eda['stars'].value_counts().values)

In [ ]:
# ratings distribution
fig, ax = plt.subplots(figsize = (14,8))
plt.bar(eda["stars"].value_counts().keys(), eda['stars'].value_counts().values)
ax.set_xlabel('Star Ratings')
ax.set_ylabel('Number of Ratings')
ax.set_title('Class Distribution')

plt.savefig('images/stars_distribution.png')

# Modeling Preparation

## Drop Neutral Reviews

Because neutral reviews aren't classified as positive or negative at this point, we can't train the model on them. We will drop the 3 star reviews for now, then manually classify them in a different dataset, at which point we will add them back into this training set with a holdout for validation

In [ ]:
df.drop(df[df['stars'] == 3].index, inplace = True)

In [ ]:
df['stars'].value_counts()

## Adding in Manually Classified Dataset

As stated above, this dataset is the 3 star reviews that were removed for the sake of training. This section adds them back in

In [ ]:
manual_classified = pd.read_csv('data/manual_classified.csv')

In [ ]:
manual_classified.shape

In [ ]:
# adding the training portion of the manually classified dataset
manual_training = manual_classified.sample(150, random_state = 42)

In [ ]:
manual_training.shape

In [ ]:
# setting the testing set of the manual classified dataset
manual_test = manual_classified[~manual_classified.index.isin(manual_training.index)]

In [ ]:
manual_test.shape

## Setting Target Variable

In [ ]:
df['target'] = df['stars'] > 3

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df['stars'].value_counts()

# Start of NLP

In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, f1_score
import string
import pickle

In [ ]:
# because of computational power, we limit the training and testing sample to 500,000 entries
sample = df.sample(500000, random_state = 42)

In [ ]:
# to ensure that we have the neutral reviews in here, we append them manually
sample = sample.append(manual_training)

In [ ]:
sample['stars'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X = sample['text']
y = sample['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#### Count Vectorizer (not used in final results)

In [ ]:
cv = CountVectorizer()
X_t_vec = cv.fit_transform(X_train)
X_t_vec  = pd.DataFrame.sparse.from_spmatrix(X_t_vec)
X_t_vec.columns = sorted(cv.vocabulary_)
X_t_vec.set_index(y_train.index, inplace=True)

In [ ]:
X_val_vec = cv.transform(X_test)
X_val_vec  = pd.DataFrame.sparse.from_spmatrix(X_val_vec)
X_val_vec.columns = sorted(cv.vocabulary_)
# X_val_vec.set_index(y_train.index, inplace=True)

In [ ]:
mnb = MultinomialNB()

mnb.fit(X_t_vec, y_train)
y_hat = mnb.predict(X_val_vec)
y_hat

## TF IDF Vectorization

In [ ]:
# stopwords

import nltk
nltk.download('stopwords')

stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``', '-', "'"]

In [ ]:
# function for removing stopwords

nltk.download('punkt')

def process_article(article):
    tokens = nltk.word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords_list]
    return stopwords_removed    

In [ ]:
# mapping funtion to X_train and X_test

processed_X_t = list(map(process_article, X_train))

processed_X_val = list(map(process_article, X_test))

In [ ]:
processed_X_t

In [ ]:
# RegEx 
## to get everything between square brackets: r'[[].*?[]]'
## to get everything that starts with a capital letter: r'[[][A-Z].*?[]]''
## to get everything that starts with a capital letter and no white space: r'[[][A-Z][a-z]*?[]]'

regex_person_reg = r"([A-Z][a-z].*?[^\s]*)\:"
regex_person_upper = r"([A-Z][^\s]*)\:"
regex_parens = r'[[][A-Z].*?[]]'


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf_data_train = vectorizer.fit_transform(X_train, y_train)
tf_idf_data_test = vectorizer.transform(X_test)

In [ ]:
# pickle vectorizer

filename = 'pickle/tfidf_vectorizer'
pickle.dump(vectorizer, open(filename, 'wb'))

In [ ]:
# load vectorizer

nb_train_preds = pickle.load(open('pickle/tfidf_vectorizer', 'rb'))

# Start of Modeling

## Baseline Modeling - NB and RF

In [ ]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=100)

In [ ]:
nb_classifier.fit(tf_idf_data_train, y_train)
nb_train_preds = nb_classifier.predict(tf_idf_data_train)
nb_test_preds = nb_classifier.predict(tf_idf_data_test)

In [ ]:
rf_classifier.fit(tf_idf_data_train, y_train)
rf_train_preds = rf_classifier.predict(tf_idf_data_train)
rf_test_preds = rf_classifier.predict(tf_idf_data_test)

#### Pickle Predictions - NB

In [ ]:
import pickle

# pickle Naive Bayes Classifier

filename = 'pickle/nb_classifier'
pickle.dump(nb_classifier, open(filename, 'wb'))

In [ ]:
# pickle Naive Bayes Classifier train predictions

filename = 'pickle/nb_train_preds'
pickle.dump(nb_train_preds, open(filename, 'wb'))

In [ ]:
# pickle Naive Bayes Classifier test predictions

filename = 'pickle/nb_test_preds'
pickle.dump(nb_test_preds, open(filename, 'wb'))

#### Pickle Preds - RF

In [ ]:
# pickle Random Forest Classifier

filename = 'pickle/rf_classifier'
pickle.dump(rf_classifier, open(filename, 'wb'))

In [ ]:
# pickle Random Forest Classifier train predictions

filename = 'pickle/rf_train_preds'
pickle.dump(rf_train_preds, open(filename, 'wb'))

In [ ]:
# pickle Random Forest Classifier test predictions

filename = 'pickle/rf_test_preds'
pickle.dump(rf_test_preds, open(filename, 'wb'))

#### Load Preds

In [ ]:
# load nb train preds

nb_train_preds = pickle.load(open('pickle/nb_train_preds', 'rb'))

In [ ]:
# load nb test preds

nb_test_preds = pickle.load(open('pickle/nb_test_preds', 'rb'))

In [ ]:
# load rf train preds

rf_train_preds = pickle.load(open('pickle/rf_train_preds', 'rb'))

In [ ]:
# load nb test preds

rf_test_preds = pickle.load(open('pickle/rf_test_preds', 'rb'))

### Prediction Metrics

In [ ]:
nb_train_score = f1_score(y_train, nb_train_preds)
nb_test_score = f1_score(y_test, nb_test_preds)
rf_train_score = f1_score(y_train, rf_train_preds)
rf_test_score = f1_score(y_test, rf_test_preds)

print("Multinomial Naive Bayes")
print("Training F1: {:.4} \t\t".format(nb_train_score))
print("")
print("Test F1: {:.4} \t\t".format(nb_test_score))
print('-'*70)
print("")
print('Random Forest')
print("Training F1: {:.4} \t\t".format(rf_train_score))
print("")
print("Test F1: {:.4} \t\t".format(nb_test_score))

### NB and RF Modeling Review
Based on what we see here, these models both seem pretty good! A relatively steep drop off from the training score and the testing score is somwhat concerning on the Random Forest model, though. Could be overfit.

One thing about the metric we're using, F1: I thought that the F1 score would be pretty relevant because we aren't too concerned about false positives and false negatives, like we would be if we were predicting the chance of a disease or something like that. We do have somewhat of a class imbalance as well, with much more of the positive reviews than the negative reviews. Thus, F1 score is a decent metric to use.

### Baseline Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 500)
lr.fit(tf_idf_data_train, y_train)
lr_train_preds = lr.predict(tf_idf_data_train)
lr_test_preds = lr.predict(tf_idf_data_test)

In [ ]:
# Pickle lr model and predictions

filename = 'pickle/lr_classifier'
pickle.dump(lr, open(filename, 'wb'))

filename = 'pickle/lr_train_preds'
pickle.dump(lr_train_preds, open(filename, 'wb'))

filename = 'pickle/lr_test_preds'
pickle.dump(lr_test_preds, open(filename, 'wb'))

#### Load LR Pickle

In [ ]:
# load lr pickle

lr_train_preds = pickle.load(open('pickle/lr_train_preds', 'rb'))

lr_test_preds = pickle.load(open('pickle/lr_test_preds', 'rb'))

In [ ]:
lr_train_score = f1_score(y_train, lr_train_preds)
lr_test_score = f1_score(y_test, lr_test_preds)

print("Logistic Regression")
print("Training F1: {:.4} \t\t".format(lr_train_score))
print("Test F1: {:.4} \t\t".format(lr_test_score))

### Logistic Regression Model Review
Great scores on both training and especially testing. I will move forward from here by using Logistic Regression.

## Grid Search

In [ ]:
params = {'solver': ['newton-cg'],
          'penalty': ['l2', 'elasticnet'], 
          'tol': [0.5, 1.0, 2.0],
          'C': [1.0, 2.0, 3.0]}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(LogisticRegression(), param_grid = params)
grid.fit(tf_idf_data_train, y_train)

In [ ]:
grid.cv_results_

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_tuned = LogisticRegression(max_iter = 500, C = 3, penalty = 'l2', solver = 'newton-cg', tol = 1)
lr_tuned.fit(tf_idf_data_train, y_train)
lr_tuned_train_preds = lr_tuned.predict(tf_idf_data_train)
lr_tuned_test_preds = lr_tuned.predict(tf_idf_data_test)

In [ ]:
# Pickle tuned lr model and predictions

filename = 'pickle/lr_classifier_tuned'
pickle.dump(lr_tuned, open(filename, 'wb'))

filename = 'pickle/lr_tuned_train_preds'
pickle.dump(lr_tuned_train_preds, open(filename, 'wb'))

filename = 'pickle/lr_tuned_test_preds'
pickle.dump(lr_tuned_test_preds, open(filename, 'wb'))

In [ ]:
# Load tuned lr pickle

# load lr pickle

lr_tuned_train_preds = pickle.load(open('pickle/lr_tuned_train_preds', 'rb'))

lr_tuned_test_preds = pickle.load(open('pickle/lr_tuned_test_preds', 'rb'))

### Tuned Logistic Regression Results

In [ ]:
lr_train_score = f1_score(y_train, lr_tuned_train_preds)
lr_test_score = f1_score(y_test, lr_tuned_test_preds)

print("Logistic Regression")
print("Training F1: {:.4} \t\t".format(lr_train_score))
print("Test F1: {:.4} \t\t".format(lr_test_score))

As we can see, slightly better results here after tuning the model. I ran through this grid search multiple times with different parameters (although it is not shown since I went back into the original parameter cell and altered it based on previous grid search results). I think this is a good enough tuned model to move forward with and test our neutral reviews on. 

## Predicting on Manually Classified Neutrals
Some deviations here from the presentation metrics because I added in more entries to the testing set

In [ ]:
X_val = manual_test['text']
y_val = manual_test['target']

In [ ]:
tf_idf_data_val = vectorizer.transform(X_val)

Baseline LR model

In [ ]:
# baseline lr validation
lr = pickle.load(open('pickle/lr_classifier', 'rb'))

In [ ]:
lr_val_preds = lr.predict(tf_idf_data_val)

In [ ]:
lr_val_score = f1_score(y_val, lr_val_preds)

print("Val F1: {:.4} \t\t".format(lr_val_score))

Tuned LR model

In [ ]:
# tuned lr validation
lr_tuned = pickle.load(open('pickle/lr_classifier_tuned', 'rb'))

In [ ]:
lr_tuned_val_preds = lr_tuned.predict(tf_idf_data_val)

In [ ]:
lr_tuned_val_score = f1_score(y_val, lr_tuned_val_preds)

print("Val F1: {:.4} \t\t".format(lr_val_score))

Basically no difference between the original baseline Logistic Regression model and the tuned Logistic Regression model. Either way, the models are predicting correctly around 73% of the time. A large portion of this is probably because of how few entries we have in the Neutral Reviews validation dataset, so the tuning isn't being fully put to use here. However, we cannot say that with absolute certainty. 

# Conclusions

In conclusion to all of this, we can see that generally speaking, Logistic Regression works the best for NLP. Of course, more neutral review data is needed for both training and validation, as adding just ~10 more entries to the validation data increased the F1 score by 5-6%. Also, the ability to separate between domains (restaurants, services, hotels, etc.) will give a huge boost to the usability of the model. 

##### Conclusion F1 Scores
>
> <b>Baseline Logistic Regression:</b>
>
> Training F1: 0.9739
>
> Test F1: 0.9697

> <b>Tuned Logistic Regression:</b>
>
> Training F1: 0.978 		
>
> Test F1: 0.9706 

> <b>Neutral Review Validation (Neutral Reviews Only):</b>
>
> Val F1: 0.7342

As of now, we have a model in which we can enter a text review, and retrieve a prediction as to whether that text review is positive or negative. However, there is not a complete business use for this until we can retrieve the predicted positive or negative as well as tokenized words to make it faster to gain insights from, which was the point of the project. Thus, I am working on deploying a webapp through Streamlit in which a user can enter the text of a reviews, and retrieve both the predicted positive or negative rating, and the tokenized words in the review, making for a quick way to gain insights. 

Below, I have an example of a Webapp I am in the progress of making. Hopefully, it can be finished and deployed soon. 

## Streamlit Webapp (In Progress)

In [3]:
#!pip install streamlit scikit-learn joblib

In [ ]:
# pip install --upgrade protobuf

In [5]:
import streamlit as st
#import joblib.os

#import spacy

def main():
    '''Review Classifier App with Streamlit'''
    st.title('Review Sentiment Classifier ML App')
    
#if __name__ == '__main__':
    main()

ImportError: DLL load failed while importing lib: The specified module could not be found.

In [ ]:
# https://www.youtube.com/watch?v=bEOiYF1a6Ak

import streamlit as st 
import joblib,os
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_sm')
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use("Agg")
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


# Vectorizer
review_vectorizer = open('pickle/tfidf_vectorizer', 'rb')
review_cv = joblib.load(review_vectorizer)

# Load Models
def load_prediction_models(model_file):
    loaded_models = joblib.load(open(os.path.join(model_file), 'rb'))
    return loaded_models

def get_keys(val, my_dict):
    for key, value in my_dict.items():
        if val == value: 
            return key

# main classifier function
def main():
    '''Review Classifier App with Streamlit'''
    st.title('Review Sentiment Classifier ML App')
    st.subheader('NLP and ML App with Streamlit')
    
    activities = ['Prediction', 'NLP']
    
    choice = st.sidebar.selectbox('Choose Activity', activities)
    
    if choice == 'Prediction':
        st.info('Prediction with ML')
        
        review_text = st.text_area('Enter Text', 'Type Here')
        all_ml_models = ['Logistic Regression', 'NB']
        model_choice = st.selectbox('Choose ML Model', all_ml_models)
        prediction_labels = {'Negative': 0, 'Positive': 1}
        if st.button('Classify'):
            st.text('Original Test ::\n{}'.format(review_text))
            vect_text = review_cv.transform([review_text]).toarray()
            if model_choice == 'LR': 
                predictor = load_prediction_models('pickle/lr_classifier_tuned')
                prediction = predictor.predict(vect_text)
                st.write(prediction)
                final_result = get_keys(prediction, prediction_labels)
                st.success(final_result)
        
        # 19:05 video timestamp
        
        
    if choice == 'NLP':
        st.info('Natural Language Processing')
    
    
if __name__ == '__main__':
    main()

In [ ]:
import streamlit as st 
import joblib,os
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_sm')
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use("Agg")
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# load Vectorizer For Gender Prediction
news_vectorizer = open("pickle/tfidf_vectorizer","rb")
news_cv = joblib.load(news_vectorizer)

def load_prediction_models(model_file):
	loaded_model = joblib.load(open(os.path.join(model_file),"rb"))
	return loaded_model

# Get the Keys
def get_key(val,my_dict):
	for key,value in my_dict.items():
		if val == value:
			return key

def main():
	"""Review Classifier"""
	st.title("Review Classifier")
	# st.subheader("ML App with Streamlit")
	html_temp = """
	<div style="background-color:blue;padding:10px">
	<h1 style="color:white;text-align:center;">Streamlit ML App </h1>
	</div>
	"""
	st.markdown(html_temp,unsafe_allow_html=True)

	activity = ['Prediction','NLP']
	choice = st.sidebar.selectbox("Select Activity",activity)


	if choice == 'Prediction':
		st.info("Prediction with ML")

		news_text = st.text_area("Enter News Here","Type Here")
		all_ml_models = ["LR","NB"]
		model_choice = st.selectbox("Select Model",all_ml_models)

		prediction_labels = {'negative': 0,'positive': 1}
		if st.button("Classify"):
			st.text("Original Text::\n{}".format(news_text))
			vect_text = news_cv.transform([news_text]).toarray()
			if model_choice == 'LR':
				predictor = load_prediction_models("pickle/lr_classifier_tuned")
				prediction = predictor.predict(vect_text)
				# st.write(prediction)
			elif model_choice == 'NB':
				predictor = load_prediction_models("pickle/nb_classifier")
				prediction = predictor.predict(vect_text)
				# st.write(prediction)
			final_result = get_key(prediction,prediction_labels)
			st.success("Reviewgorized as:: {}".format(final_result))

	if choice == 'NLP':
		st.info("Natural Language Processing of Text")
		raw_text = st.text_area("Enter News Here","Type Here")
		nlp_task = ["Tokenization","Lemmatization","NER","POS Tags"]
		task_choice = st.selectbox("Choose NLP Task",nlp_task)
		if st.button("Analyze"):
			st.info("Original Text::\n{}".format(raw_text))

			docx = nlp(raw_text)
			if task_choice == 'Tokenization':
				result = [token.text for token in docx ]
			elif task_choice == 'Lemmatization':
				result = ["'Token':{},'Lemma':{}".format(token.text,token.lemma_) for token in docx]
			elif task_choice == 'NER':
				result = [(entity.text,entity.label_)for entity in docx.ents]
			elif task_choice == 'POS Tags':
				result = ["'Token':{},'POS':{},'Dependency':{}".format(word.text,word.tag_,word.dep_) for word in docx]

			st.json(result)

		if st.button("Tabulize"):
			docx = nlp(raw_text)
			c_tokens = [token.text for token in docx ]
			c_lemma = [token.lemma_ for token in docx ]
			c_pos = [token.pos_ for token in docx ]

			new_df = pd.DataFrame(zip(c_tokens,c_lemma,c_pos),columns=['Tokens','Lemma','POS'])
			st.dataframe(new_df)


		if st.checkbox("WordCloud"):
			c_text = raw_text
			wordcloud = WordCloud().generate(c_text)
			plt.imshow(wordcloud,interpolation='bilinear')
			plt.axis("off")
			st.pyplot()









	st.sidebar.subheader("About")